### 목표 설정
- 폐암 데이터를 활용해서 이진 분류 신경망 구현
- 이진 분류 개념을 이해한다.

In [1]:
# 드라이브 마운트(연동)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/딥러닝

/content/drive/MyDrive/Colab Notebooks/딥러닝


In [3]:
!pwd

/content/drive/MyDrive/Colab Notebooks/딥러닝


In [4]:
# 필요한 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
# 데이터 불러오기
data = pd.read_csv('data/ThoraricSurgery.csv', header = None)
data
# 컬럼명이 삭제되어서 0번 인덱스가 컬럼명으로 올라갔다 -> header=None

# 현재 데이터는 컬럼명이 별도로 존재하지 않는다.
# 이 상태로 데이터를 불러오면 첫번째 행이 컬럼 명으로 불러와진다.
# header 옵션을 사용해서 새로 불러와보자
# header : 데이터를 불러오면서 컬럼명을 설정해주는 키워드
# (None)으로 설정한 경우 : 인덱스 번호가 컬럼명으로 출력된다.
# 데이터 정보 : 0 ~ 16 컬럼 -> 속성(특성) : 종양 유형, 폐활량, 고통정도 등등
# 17번 컬럼 : 정답 클래스(수술 후 생존 결과) - 0(사망) / 1(생존)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,293,1,3.80,2.80,0,0,0,0,0,0,12,0,0,0,1,0,62,0
1,1,2,2.88,2.16,1,0,0,0,1,1,14,0,0,0,1,0,60,0
2,8,2,3.19,2.50,1,0,0,0,1,0,11,0,0,1,1,0,66,1
3,14,2,3.98,3.06,2,0,0,0,1,1,14,0,0,0,1,0,80,1
4,17,2,2.21,1.88,0,0,1,0,0,0,12,0,0,0,1,0,56,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,98,6,3.04,2.40,2,0,0,0,1,0,11,0,0,0,1,0,76,0
466,369,6,3.88,2.72,1,0,0,0,1,0,12,0,0,0,1,0,77,0
467,406,6,5.36,3.96,1,0,0,0,1,0,12,0,0,0,0,0,62,0
468,25,8,4.32,3.20,0,0,0,0,0,0,11,0,0,0,0,0,58,1


In [10]:
data.info()
# 결측치 없다!
# Dtype : 정수와 실수로 이루어져 있다 -> 인코딩 필요 없다!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       470 non-null    int64  
 1   1       470 non-null    int64  
 2   2       470 non-null    float64
 3   3       470 non-null    float64
 4   4       470 non-null    int64  
 5   5       470 non-null    int64  
 6   6       470 non-null    int64  
 7   7       470 non-null    int64  
 8   8       470 non-null    int64  
 9   9       470 non-null    int64  
 10  10      470 non-null    int64  
 11  11      470 non-null    int64  
 12  12      470 non-null    int64  
 13  13      470 non-null    int64  
 14  14      470 non-null    int64  
 15  15      470 non-null    int64  
 16  16      470 non-null    int64  
 17  17      470 non-null    int64  
dtypes: float64(2), int64(16)
memory usage: 66.2 KB


### 데이터 분리
1. 문제 데이터 / 정답 데이터 분리
2. 훈련 데이터와 평가 데이터 분리

In [9]:
# 1. 문제 데이터와 정답 데이터로 분리 (iloc)
X = data.iloc[:, :17]   # X = data.iloc[:, :-1]
y = data[17]
y

0      0
1      0
2      1
3      1
4      0
      ..
465    0
466    0
467    0
468    1
469    0
Name: 17, Length: 470, dtype: int64

In [12]:
# 데이터 체크(shape)
X.shape, y.shape

((470, 17), (470,))

In [13]:
# 훈련셋과 평가셋으로 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=30)

In [14]:
# 데이터 체크(shape)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((329, 17), (141, 17), (329,), (141,))

### 신경망 구축
1. 신경망 구조 설계
2. 학습/평가 방법 설정
3. 학습 + 시각화
4. 모델 평가

In [15]:
# 재료 import
# 뼈대를 구성하는 재료
# 층 구성 재료
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

In [16]:
# 모델 구축 시작
# 뼈대 설정
model = Sequential()

# 입력층 설정
# 뉴런의 개수 = 10 / 데이터 입력 특성 개수 = ? / sigmoid 함수 사용
model.add(Dense(units=10, input_dim=17, activation='sigmoid'))

# 중간층 설정 -> 입력층에서 결과를 넘겨주기 때문에 input_dim 생략 가능
# 뉴런의 개수 = 6 / 시그모이드 함수
model.add(Dense(units=6, activation='sigmoid'))

# 뉴런의 개수 = 4 / 시그모이드 함수
model.add(Dense(units=4, activation='sigmoid'))

# 출력층 설정
# 뉴런의 개수 = 1 / 시그모이드 함수
model.add(Dense(units=1, activation='sigmoid'))
# 현재 우리가 진행하는 분석 방법은 어떤 방법인가? > 분류 > 이진 분류
# 출력층의 구조는 우리가 예측하는 분석 방법에 따라 구조가 달라진다.
# 이진 분류에서는 활성화 함수 sigmoid / 뉴런의 개수 1개 사용
# 회귀 분석에서는 활성화 함수 생략 / 뉴런의 개수 1개 사용

# 모델 요약
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                180       
                                                                 
 dense_1 (Dense)             (None, 6)                 66        
                                                                 
 dense_2 (Dense)             (None, 4)                 28        
                                                                 
 dense_3 (Dense)             (None, 1)                 5         
                                                                 
Total params: 279 (1.09 KB)
Trainable params: 279 (1.09 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
